In [ ]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.2 MB/s eta 0:00:00


In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("LucasThil/miniwob_plusplus_T5_randomized_ref2")
raw_datasets

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 60321
    })
})

In [ ]:
# select how much of the datset we want to use
coef = 0.7
raw_datasets['train'] = raw_datasets['train'].shuffle()
raw_datasets['train'] = raw_datasets["train"].select(range(int(len(raw_datasets["train"])*coef)))
raw_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
raw_datasets["validation"] = raw_datasets.pop("test")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 38001
    })
    validation: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 4223
    })
})

In [ ]:
# Add extra column regarding condensed target_labels
# We append together the (action, ref, text)
# Where action is either of type 'click' or 'type'
# ref is the targeted element number to perform the action on
# and type is the text content in case the action is 'type'
def condensed_targets(dataset, label):
  condensed_targets = []
  for row in dataset[label]:
    target_text = row['keydown_texts']
    if target_text is None:
      target_text = ""
    target = '{' + row['actions'] + ', ' + str(row['refs']) + ', ' + target_text + '}'
    condensed_targets.append(target)
  return condensed_targets

raw_datasets['train'] = raw_datasets['train'].add_column("targets", condensed_targets(raw_datasets, 'train'))
raw_datasets['train'] = raw_datasets['train'].remove_columns(['actions', 'refs', 'keydown_texts', 'instruction', 'html_snippets'])
raw_datasets['validation'] = raw_datasets['validation'].add_column("targets", condensed_targets(raw_datasets, 'validation'))
raw_datasets['validation'] = raw_datasets['validation'].remove_columns(['actions', 'refs', 'keydown_texts', 'instruction', 'html_snippets'])

Flattening the indices:   0%|          | 0/38001 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4223 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 38001
    })
    validation: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 4223
    })
})

In [ ]:
print(raw_datasets['train']['history_episodes'][0])
print(raw_datasets['train']['targets'][0])

{click,8,}{click,12,}{click,1,}{click,4,} Select words similar to like, savory, afraid, home and click Submit.  <body ref="13"><div id="wrap" ref="4"><div id="area" ref="9"><div id="boxes" ref="8"><label ref="2"><input type="checkbox" id="ch0" ref="5" value="false"></input><t ref="-211" text="terrified"></t></label><label ref="3"><input type="checkbox" id="ch1" ref="7" value="false"></input><t ref="-212" text="mistaken"></t></label><label ref="10"><input type="checkbox" id="ch2" ref="12" value="true"></input><t ref="-213" text="adore"></t></label><label ref="11"><input type="checkbox" id="ch3" ref="1" value="true"></input><t ref="-214" text="homes"></t></label><label ref="13"><input type="checkbox" id="ch4" ref="4" value="true"></input><t ref="-215" text="murder"></t></label><label ref="9"><input type="checkbox" id="ch5" ref="8" value="true"></input><t ref="-216" text="delicious"></t></label></div><button id="subbtn" classes="secondary-action" ref="6" text="submit"></button></div></div

In [ ]:
from transformers import AutoTokenizer
from transformers import T5Tokenizer, T5Model

checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#tokenizer = T5Tokenizer.from_pretrained(checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [doc for doc in examples["history_episodes"]]
    targets = [doc for doc in examples["targets"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_input_length, truncation=True)

    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/38001 [00:00<?, ? examples/s]

Map:   0%|          | 0/4223 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=1fcb86b192b0ad7fdaaa0332845959f41e005f3c4d02d870572ab84882e8f71a
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="T5_flan_small_rouge_ref2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/T5_flan_small_rouge_ref2 is already a clone of https://huggingface.co/LucasThil/T5_flan_small_rouge_ref2. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.000000,nan,0.018300,0.001200,0.018300,0.018200,17.620600
2,0.000000,nan,0.018300,0.001200,0.018300,0.018200,17.620600


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:
import torch
torch.cuda.empty_cache()
